In [4]:
import pandas as pd
import numpy as np
from py2neo import Graph
from py2neo import Node, Relationship

In [5]:
node_properties = pd.read_pickle("All_node_properties.pkl")
edge_properties = pd.read_pickle("All_edge_properties.pkl")
connections = pd.read_pickle("All_connections.pkl")
edge_type = pd.read_pickle("All_edge_types.pkl")
node_type = pd.read_pickle("All_node_types.pkl")

In [7]:
# Connect to Neo4J Graph
graph = Graph("bolt://localhost:7687", user="neo4j", password="team1")

# Clear graph for inserts
graph.delete_all()
graph.schema.create_uniqueness_constraint("Person","name")

In [8]:
# Creating the relationship between Nodes
for i in range(len(connections)):
    temp_prop_a = node_properties.loc[node_properties['Node_Primary_Key'] == connections['Node_Property_Key'][i]].reset_index()
    temp_type_a = node_type.loc[node_type['Primary_Key'] == temp_prop_a['Node_Type_Key'][0]].reset_index()
    node_name_a = temp_prop_a.loc[temp_prop_a['Property_Name'] == 'Name'].reset_index()
    temp_node_a = Node(temp_type_a['Node_Type'][0], name = str(node_name_a['Property'][0]))
    for j in range(len(temp_prop_a)):
        if temp_prop_a['Property_Name'][j] == 'Date':
            temp = int(temp_prop_a['Property'][j][-4:])
            temp_node_a[temp_prop_a['Property_Name'][j]] = temp
        elif temp_prop_a['Property_Name'][j] != 'Name' and isinstance(temp_prop_a['Property'][j], str):
            temp_node_a[temp_prop_a['Property_Name'][j]] = str(temp_prop_a['Property'][j])
        elif isinstance(temp_prop_a['Property'][j], float):
            if not np.isnan(temp_prop_a['Property'][j]):
                temp_node_a[temp_prop_a['Property_Name'][j]] = temp_prop_a['Property'][j]
        elif isinstance(temp_prop_a['Property'][j], int):
            if not np.isnan(temp_prop_a['Property'][j]):
                temp_node_a[temp_prop_a['Property_Name'][j]] = temp_prop_a['Property'][j]

    temp_prop_b = node_properties.loc[node_properties['Node_Primary_Key'] == connections['Node_Property_Key_2'][i]].reset_index()
    temp_type_b = node_type.loc[node_type['Primary_Key'] == temp_prop_b['Node_Type_Key'][0]].reset_index()
    node_name_b = temp_prop_b.loc[temp_prop_b['Property_Name'] == 'Name'].reset_index()
    
    if node_name_b.empty and (temp_prop_b['Node_Type_Key'][0] == 4):
        node_name_b = str(node_name_a['Property'][0]) + ' FD'
        temp_node_b = Node(temp_type_b['Node_Type'][0], name = node_name_b)
    elif node_name_b.empty and (temp_prop_b['Node_Type_Key'][0] == 6):
        node_name_b = str(temp_prop_b.loc[temp_prop_b['Property_Name'] == 'Code'].reset_index())
        node_name_b = 'Parent NAICS Code ' + node_name_b
        temp_node_b = Node(temp_type_b['Node_Type'][0], name = node_name_b)
    elif node_name_b.empty and (temp_prop_b['Node_Type_Key'][0] == 5):
        node_name_b = str(node_name_a['Property'][0]) + ' Social Media'
        temp_node_b = Node(temp_type_b['Node_Type'][0], name = node_name_b)    
    else:
        temp_node_b = Node(temp_type_b['Node_Type'][0], name = str(node_name_b['Property'][0]))
        
    for j in range(len(temp_prop_b)):
        if temp_prop_b['Property_Name'][j] == 'Date':
            temp = int(temp_prop_b['Property'][j][-4:])
            temp_node_b[temp_prop_b['Property_Name'][j]] = temp
        elif temp_prop_b['Property_Name'][j] != 'Name' and isinstance(temp_prop_b['Property'][j], str):
            temp_node_b[temp_prop_b['Property_Name'][j]] = str(temp_prop_b['Property'][j])
        elif isinstance(temp_prop_b['Property'][j], float):
            if not np.isnan(temp_prop_b['Property'][j]):
                temp_node_b[temp_prop_b['Property_Name'][j]] = temp_prop_b['Property'][j]
        elif isinstance(temp_prop_b['Property'][j], int):
            if not np.isnan(temp_prop_b['Property'][j]):
                temp_node_b[temp_prop_b['Property_Name'][j]] = temp_prop_b['Property'][j]
            
    edge_prop = edge_properties.loc[edge_properties['Edge_Primary_Key'] == connections['Edge_Property_Key'][i]].reset_index()
    temp_edge_type = edge_type.loc[edge_type['Edge_Primary_Key'] == edge_prop['Edge_Type_Key'][0]].reset_index()
    relationship = str(temp_edge_type['Edge_Type'][0])
    edge_init = Relationship(temp_node_a, relationship, temp_node_b)
    for j in range(len(edge_prop)):
        if edge_prop['Property_Name'][j] == '% Overlap':
            edge_init[edge_prop['Property_Name'][j]] = int(edge_prop['Property'][j])
        else:
            edge_init[edge_prop['Property_Name'][j]] = str(edge_prop['Property'][j])
    
    tx = graph.begin()
    tx.merge(temp_node_a,temp_type_a['Node_Type'][0],"name") #node,label,primary key
    tx.merge(temp_node_b,temp_type_b['Node_Type'][0],"name") #node,label,pirmary key
    tx.merge(edge_init)
    tx.commit()